In [1]:
!pip install langchain_community langchainhub chromadb langchain langchain-openai

In [2]:
from google.colab import userdata
import os
os.environ['GOOGLE_API_KEY'] = userdata.get('GOOGLE_API_KEY')
os.environ['HF_TOKEN'] = userdata.get('HF_TOKEN')
os.environ['MISTRALAI_API_KEY'] = userdata.get('MISTRALAI_API_KEY')

In [3]:
from langchain_community.document_loaders.csv_loader import CSVLoader

loader = CSVLoader(file_path="/content/legal_cases.csv")

# Load all documents
documents = loader.load()
print(documents)

[Document(metadata={'source': '/content/legal_cases.csv', 'row': 0}, page_content='case_id: 1\ncase_name: Imax Corporation vs E-City Entertainment (I) Pvt. Limited ... on 24 October, 2024\nFacts: 1. The Petitioner- IMAX Corporation (for short "IMAX") , a Canadian Corporation, claiming to be the world\'s leading entertainment technology company, specializing in motion picture technology and large format motion picture presentations, having its registered office at 2525 Speakman Drive, Mississauga, Ontario, Canada, holding three Awards in its favour, pursuant to the arbitration proceedings, being concluded between it and E-City Entertainment (I) Pvt. Ltd. (Respondent No.1) before the International Court of Arbitration of the International Chamber of Commerce in London, has approached this Court, by filing the present Arbitration Petition, invoking Section 47, 48 and 49 of the Arbitration and Conciliation Act, 1996 (for short "the Act of 1996"), seeking a declaration that the three Arbitr

In [4]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 200)
splits = text_splitter.split_documents(documents)

In [5]:
print(splits[0])
print(splits[1])
print(splits[2])

page_content='case_id: 1
case_name: Imax Corporation vs E-City Entertainment (I) Pvt. Limited ... on 24 October, 2024
Facts: 1. The Petitioner- IMAX Corporation (for short "IMAX") , a Canadian Corporation, claiming to be the world's leading entertainment technology company, specializing in motion picture technology and large format motion picture presentations, having its registered office at 2525 Speakman Drive, Mississauga, Ontario, Canada, holding three Awards in its favour, pursuant to the arbitration proceedings, being concluded between it and E-City Entertainment (I) Pvt. Ltd. (Respondent No.1) before the International Court of Arbitration of the International Chamber of Commerce in London, has approached this Court, by filing the present Arbitration Petition, invoking Section 47, 48 and 49 of the Arbitration and Conciliation Act, 1996 (for short "the Act of 1996"), seeking a declaration that the three Arbitral Awards in its favour, are enforceable under the provisions of Part II

In [6]:
print(len(splits))

1873


In [7]:
!pip install -qU langchain-mistralai

In [8]:
!pip install -qU langchain-huggingface

In [9]:
!pip install -qU langchain-chroma

In [10]:
from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

In [11]:
from langchain_chroma import Chroma

vectorstore = Chroma.from_documents(documents=splits, embedding=embeddings)

In [12]:
print(vectorstore._collection.count())

1873


In [13]:
print(vectorstore._collection.get())

{'ids': ['55066715-0031-4bdd-abc7-91733cce71e8', 'ca8f7f51-ce21-470c-a479-84e7edf1435a', '1f52f8a4-e231-4162-9b7e-431d758726ae', 'e1365eb2-5bf0-4428-bb8c-722854d94257', 'e89fd357-3fe4-4b1b-a434-ee1e8452867f', '46b4d722-ed7c-4438-b691-51b39f6d4f75', '6c5bae08-76c4-4114-9159-35806d91381d', 'e6010f92-f329-4419-99fd-60f73ccae0a9', '6c721ef8-516c-42da-bd40-06758e224633', 'b72ed171-4303-4747-a1f5-3503cd3a3f24', 'b87c9fd9-9003-4559-a518-2ec20fa34f66', '09d1df2b-2362-40fc-9ec7-662276c8aad6', 'd21ece24-70f2-47e8-bf8e-e9e70016dac7', 'e32f99f5-c984-4c47-9ad9-428ff9d8d890', '81e9cfe4-819e-4591-b527-bff0c7896803', 'c57cff98-8aac-4150-a796-fb91245099af', '23ca4968-91b0-4fd0-88e9-b158c03a2cc5', '3b338cc2-d408-478d-9f96-9d2dcb5979a8', '36ae0292-8adc-4f93-9ebd-1a51f2ccf79e', 'fb2682fd-e0e3-4352-ae77-f818482acd26', '31bbe815-48dc-4987-a07f-3b63a58ba525', 'd1709d40-4016-45e6-9f6e-c83b3f341d33', '59130485-2ae7-46fd-9fb7-11b639923703', '6acdcd1f-53a6-41f8-bf64-eb1a027312ea', 'b3679244-09e7-4746-aaca-73c5c6

In [14]:
print("\nCollection 1 - ", vectorstore._collection.get(ids=['be835d8a-6e6f-4607-9968-aae4b154412d'], include=["embeddings", "documents"]))
print("\nCollection 2 - ", vectorstore._collection.get(ids=['3b6b2aa5-7387-44ac-a353-61e53ab962c2'], include=["embeddings", "documents"]))
print("\nCollection 3 - ", vectorstore._collection.get(ids=['72d0bf4d-f223-4363-bebe-faeb5ffa48da'], include=["embeddings", "documents"]))


Collection 1 -  {'ids': [], 'embeddings': array([], dtype=float64), 'documents': [], 'uris': None, 'included': ['embeddings', 'documents'], 'data': None, 'metadatas': None}

Collection 2 -  {'ids': [], 'embeddings': array([], dtype=float64), 'documents': [], 'uris': None, 'included': ['embeddings', 'documents'], 'data': None, 'metadatas': None}

Collection 3 -  {'ids': [], 'embeddings': array([], dtype=float64), 'documents': [], 'uris': None, 'included': ['embeddings', 'documents'], 'data': None, 'metadatas': None}


In [15]:
retriever = vectorstore.as_retriever()

In [16]:
from langchain_core.prompts import PromptTemplate

In [17]:
rag_prompt = PromptTemplate(
    input_variables=["question", "context"],
    template=(
        "You are a legal assistant for Indian case law. You will answer based ONLY "
        "on the provided case excerpts.\n\n"
        "If you are not sure, say you are not sure and suggest consulting a lawyer.\n\n"
        "Context (case excerpts):\n"
        "{context}\n\n"
        "User question:\n"
        "{question}\n\n"
        "Instructions:\n"
        "- Use only the information in the context.\n"
        "- Mention the relevant case names when explaining.\n"
        "- If you add your own general interpretation, mark it clearly.\n"
        "- End with: 'This is not formal legal advice.'\n\n"
        "Answer:\n"
    ),
)


In [18]:
query = "tenant eviction without proper notice"
results = retriever.invoke(query)

In [19]:
print(results)

[Document(id='587d8471-fe61-4863-a085-446c9d21f306', metadata={'source': '/content/legal_cases.csv', 'row': 22}, page_content='"36Accordingly, had E-City fully performed its rental obligations under the Master Agreement, IMAX would have realised at least $21,222,920 in revenue over the 20 year term of the leases called for under the Master Agreement, obligations which would, by now, have 45/168 CARBP-414-18.odt been largely performed. E-city should be directed to make payment of this amount to IMAX in order to make IMAX fully whole for its breach of its obligations. In contract, IMAX fully performed and complied with its exclusivity obligations."\nUnnamed: 5: \nPetitioner\'s Arguments: \nRespondent\'s Arguments: Respondent No.4 face the accusation that as the holding Company of Respondent Nos.1, 2 and 3, it has arranged for the improper diversion of Respondent No.1 to Respondent Nos.2 and 3.'), Document(id='20c320a2-4771-42a0-9477-3ec8c6743105', metadata={'row': 19, 'source': '/content

In [20]:
!pip install langchain_google_genai

In [21]:
from langchain_google_genai import ChatGoogleGenerativeAI

In [22]:
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0.2,
    convert_system_message_to_human=True,
)


In [23]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

In [24]:
def format_context(docs):
    lines = []
    for i, d in enumerate(docs, start=1):
        m = d.metadata
        lines.append(
            f"[CASE {i}] {m.get('case_name', 'Unknown')} "
            f"({m.get('court', 'Unknown Court')}, {m.get('date', 'Unknown Date')})\n"
            f"Excerpt:\n{d.page_content}\n"
            "-----"
        )
    return "\n".join(lines)

In [25]:
rag_chain = ({"context" : retriever | format_context, "question" : RunnablePassthrough()}
             | rag_prompt
             | llm
             | StrOutputParser())

In [26]:
rag_chain.invoke("A tenant in a rented flat is being evicted without proper notice. What rights does the tenant have based on these cases?")

'Based on the provided case excerpts, there is no direct information regarding the rights of a tenant being evicted without proper notice.\n\n*   **CASE 1, CASE 2, and CASE 3** discuss a ruling in *PPL vs. Avion Hospitality Pvt. Ltd.* where a defendant may raise "appropriate contentions including raising the issue of maintainability of the Suits at the instance of the present Plaintiff." This pertains to procedural aspects of a lawsuit and does not address tenant eviction rights.\n*   **CASE 4** mentions "the subject agreement was for lease and not for sale" in the Respondent\'s Arguments. However, the Court\'s Reasoning in this case focuses on the right to object to the enforcement of an award and the grounds for such objections (e.g., incapacity of parties, invalidity of agreement), rather than the rights of a tenant in an eviction scenario.\n\nTherefore, the provided case excerpts do not contain information that directly addresses the rights of a tenant being evicted without proper 

In [27]:
from langchain_core.runnables import RunnableLambda

In [28]:
def print_prompt(prompt_text):
  print("Prompt - ", prompt_text)
  return prompt_text

In [29]:
rag_chain_with_print = ({"context" : retriever | format_context, "question" : RunnablePassthrough()}
             | rag_prompt
             | RunnableLambda(print_prompt)
             | llm
             | StrOutputParser())

In [30]:
rag_chain_with_print.invoke("What limitation period did the Court apply for filing a petition to enforce the foreign arbitral awards in IMAX Corporation vs E-City Entertainment (I) Pvt. Ltd., and was the petition held to be within time?")

Prompt -  text='You are a legal assistant for Indian case law. You will answer based ONLY on the provided case excerpts.\n\nIf you are not sure, say you are not sure and suggest consulting a lawyer.\n\nContext (case excerpts):\n[CASE 1] Unknown (Unknown Court, Unknown Date)\nExcerpt:\ncase_id: \ncase_name: \nFacts: \nIssues: (F) The necessity and validity for impleadment of Respondent Nos.2 to 4 in the present Petition, seeking enforcement and execution of the foreign awards?\r\n\r\n                                 51/168                 CARBP-414-18.odt\r\n\r\n  (G)      Whether the invocation of arbitration by IMAX Limited,\r\nwhich has merged into IMAX Corporation, was invalid? I shall deal with the above issues sequentially.\n-----\n[CASE 2] Unknown (Unknown Court, Unknown Date)\nExcerpt:\ncase_id: \ncase_name: \nFacts: \nIssues: (A) In case of the foreign awards, whether the award holder (IMAX Corporation) is entitled to apply for recognition and enforcement of the Award as deemed

'Based on the provided case excerpts:\n\nThe issue of whether the petition filed by E-City Corporation for enforcement, recognition, and execution was filed beyond the prescribed period of limitation as contemplated under the Limitation Act, 1963, was raised in the case (as per CASE 2, Issue (C)).\n\nHowever, the provided excerpts do not specify what limitation period the Court applied, nor do they state whether the petition was ultimately held to be within time or beyond time.\n\nThis is not formal legal advice.'

In [31]:
rag_chain.invoke("What limitation period did the Court apply for filing a petition to enforce the foreign arbitral awards in IMAX Corporation vs E-City Entertainment (I) Pvt. Ltd., and was the petition held to be within time?")

'Based on the provided case excerpts:\n\n[CASE 2] raises the issue: "(C) Whether the petition filed by E-City Corporation for enforcement, recognition and execution is filed beyond the prescribed period of limitation as contemplated under the Limitation Act, 1963?"\n\nThe excerpts indicate that the Limitation Act, 1963, was the law contemplated for determining the prescribed period for filing the petition. However, the provided excerpts do not state what specific limitation period the Court applied, nor do they state whether the petition was ultimately held to be within time.\n\nI am not sure what specific limitation period was applied or if the petition was held to be within time, as this information is not present in the provided excerpts. I suggest consulting a lawyer for further clarification.\n\nThis is not formal legal advice.'